In [ ]:
# IMPORTS
from tensorflow.keras import layers, models
import time
import ast
import tensorflow as tf
from copy import deepcopy
import random
from numpy import exp

In [ ]:
# GLOBAL VARIABLES
show = True
predictor_data_filename = "/home/data.txt"
input_size = 18

In [ ]:
# PREDICTOR MODEL
class Predictor:
    def __init__(self, n_layers, keras_loss_fun, hyperparameters, show_summary=False):
        # Hyperparameters:
        # List of dictionaries (one for each layer)
        model = models.Sequential()
        # Create input and hidden layers
        model.add(layers.Dense(hyperparameters[0]['size'], input_dim=input_size, activation=hyperparameters[0]['activation']))
        for i in range(1, n_layers):
            model.add(layers.Dense(hyperparameters[i]['size'], activation=hyperparameters[i]['activation']))

        # Create output layer (accuracy, test_latency)
        model.add(layers.Dense(2))  # No activation function, since it is a regression problem

        model.compile(optimizer='adam',
                      loss=keras_loss_fun,
                      metrics=['accuracy'])

        if show_summary:
            model.summary()

        self.model = model

    def train(self, train_data, train_labels, test_data, test_labels):

        history = self.model.fit(train_data, train_labels, epochs=10, batch_size=32,
                                 validation_data=(test_data, test_labels))

        start_time = time.time()
        test_loss, test_acc = self.model.evaluate(test_data, test_labels)
        end_time = time.time()
        test_time = end_time - start_time
        return test_acc, test_time

In [ ]:
# TRAIN AND TEST DATA
def _get_predictor_data(filename):
    hyperparameters, outputs = _parse_predictor_data_file(filename)
    hyperparameters, outputs = _preprocess_predictor_data(hyperparameters, outputs)

    dataset_size = len(hyperparameters)
    train_percentage = 0.7
    train_size = round(dataset_size * train_percentage)

    dataset_indexes = list(range(0, dataset_size))
    random.shuffle(dataset_indexes)
    train_sample_indexes = random.sample(dataset_indexes, train_size)
    test_sample_indexes = [index for index in dataset_indexes if index not in train_sample_indexes]


    train_samples = list(map(list, zip(*[(hyperparameters[i], outputs[i]) for i in train_sample_indexes])))
    train_data = train_samples[0]
    train_labels = train_samples[1]

    test_samples = list(map(list, zip(*[(hyperparameters[i], outputs[i]) for i in test_sample_indexes])))
    test_data = test_samples[0]
    test_labels = test_samples[1]



    return train_data, train_labels, test_data, test_labels


activation_mappings = {
    'linear': 0,
    'relu': 1,
    'sigmoid': 5,
    'tanh': 6,
    'softmax': 8,
    'exponential': 10,
}

padding_mappings = {
    'valid': 0,
    'same': 1,
}


def _preprocess_predictor_data(inputs, outputs):
    processed_inputs = []
    for model in inputs:
        processed_model = []
        for layer_type, params in model.items():  # Iterate over all layers
            if 'conv' in layer_type:
                processed_model.append(activation_mappings[params['activation']])
                processed_model.append(padding_mappings[params['padding']])
                processed_model.append(params['kernel_size'])
                processed_model.append(params['filters'] / 10)  #  Make it around the same size as the others
            else:
                processed_model.append(padding_mappings[params['padding']])
                processed_model.append(params['strides'])
                processed_model.append(params['pool_size'])
        if show:
            print("\nModel = {}".format(model))
            print("\n--> Processed model = {}\n".format(processed_model))
        processed_inputs.append(processed_model)
    return processed_inputs, outputs


def _parse_predictor_data_file(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()

    hyperparameters = []
    outputs = []  # Output  = (accuracy, latency)
    i = 0
    for line in lines:
        i += 1
        line = line.strip()
        try:
          result = ast.literal_eval(line)
          hyperparameters.append(result['HP'])
          outputs.append((result['Accuracy'], result['Latency: ']))
        except:
           if show:
                print("Error parsing data line ({})\n".format(i))

    return hyperparameters, outputs



# GET PREDICTOR DATA
training_data, training_labels, testing_data, testing_labels = _get_predictor_data(predictor_data_filename)

Error parsing data line (94)

Error parsing data line (401)

Error parsing data line (543)

Error parsing data line (625)

Error parsing data line (634)

Error parsing data line (725)


Model = {'conv_1': {'activation': 'relu', 'kernel_size': 3, 'padding': 'valid', 'filters': 32}, 'pool_1': {'pool_size': 2, 'strides': 2, 'padding': 'valid'}, 'conv_2': {'activation': 'relu', 'kernel_size': 3, 'padding': 'valid', 'filters': 64}, 'pool_2': {'pool_size': 2, 'strides': 2, 'padding': 'valid'}, 'conv_3': {'activation': 'relu', 'kernel_size': 3, 'padding': 'valid', 'filters': 64}}

--> Processed model = [1, 0, 3, 3.2, 0, 2, 2, 1, 0, 3, 6.4, 0, 2, 2, 1, 0, 3, 6.4]


Model = {'conv_1': {'activation': 'tanh', 'kernel_size': 3, 'padding': 'valid', 'filters': 13}, 'pool_1': {'pool_size': 2, 'strides': 2, 'padding': 'valid'}, 'conv_2': {'activation': 'relu', 'kernel_size': 3, 'padding': 'same', 'filters': 92}, 'pool_2': {'pool_size': 2, 'strides': 2, 'padding': 'valid'}, 'conv_3': {'activation': 're

In [ ]:
# PREDICTOR HELPER

predictor_loss_set = [
    'mean_squared_error',
    'mean_absolute_error',
    tf.keras.losses.Huber(),
    tf.keras.losses.LogCosh()
]

predictor_n_layer_set = range(5, 10)

predictor_layer_hp_set = {
    'activation': [
        'relu',
        'selu',
        'tanh',
        'linear',
        'swish',
    ],
    'size': [20, 40, 60, 80, 100]
}


def get_predictor_layer_default_hyperparameters():
    return {
        'activation': random.sample(predictor_layer_hp_set['activation'], 1)[0],
        'size': random.sample(predictor_layer_hp_set['size'], 1)[0]
    }


def get_default_predictor(n_layers, loss):
    return {
        "keras_loss_fun": loss,
        "n_layers": n_layers,
        "hyperparameters": [get_predictor_layer_default_hyperparameters() for _ in range(0, n_layers)]
    }

def predictor_objective(params):
    n_layers = params["n_layers"]
    keras_loss_fun = params["keras_loss_fun"]
    hyperparameters = params["hyperparameters"]
    predictor = Predictor(n_layers, keras_loss_fun, hyperparameters)

    test_acc, test_time = predictor.train(training_data, training_labels, testing_data, testing_labels)

    obj_value = test_acc**2 / test_time
    if show:
        print("\n", "-" * 8, "test_accuracy: {} #### test_latency: {} #### objective: {}"
                    .format(test_acc,test_time, obj_value), "-" * 8)

    return obj_value


def predictor_get_random_neighbouring_solution(old_solution):
    solution = deepcopy(old_solution)
    n_layers = solution["n_layers"]

    # Change the size of two layers
    change_size_layers = random.sample(range(0, n_layers), 2)
    for change_size_layer in change_size_layers:
        new_size = random.sample(predictor_layer_hp_set["size"], 1)[0]
        solution["hyperparameters"][change_size_layer]["size"] = new_size
    # Change activation of one layer
    change_activation_layer = random.sample(range(0, n_layers), 1)[0]
    new_activation = random.sample(predictor_layer_hp_set["activation"], 1)[0]
    solution["hyperparameters"][change_activation_layer]["activation"] = new_activation

    return solution


In [ ]:
# METAHEURISTICS: SIMULATED ANNEALING
def simulated_annealing(initial_solution, neighbour_gen_fun, objective_fun, iterations, c, seed=-1):
    # Initialization phase
    if seed >= 0:
        random.seed = seed
    current_solution = initial_solution
    current_eval = objective_fun(current_solution)
    # Optimization phase
    for i in range(iterations):
        next_solution = neighbour_gen_fun(current_solution)
        next_eval = objective_fun(next_solution)
        if next_eval > current_eval or current_eval == 0 or _accept_worse_solution(c, i+1, current_eval, next_eval):
            current_solution = next_solution
            current_eval = next_eval
            if show:
                print("\n", "#", "Accepted new solution")
    return current_solution, current_eval


def _accept_worse_solution(c, interval, curr_val, next_val):
    return random.random() < exp(interval * (next_val - curr_val) / (c * curr_val))

In [ ]:
# OPTIMIZER

def _predictor_sa_optimization(n_layers, loss_fun):
    c = 1
    iterations_per_conf = 50
    seed = -1
    return simulated_annealing(get_default_predictor(n_layers, loss_fun),
                               predictor_get_random_neighbouring_solution,
                               predictor_objective,
                               iterations_per_conf,
                               c,
                               seed)

def optimize_predictor(optimize_fun):
    best_results = {}

    for n_layers in predictor_n_layer_set:
        for loss_fun in predictor_loss_set:
            model, eval = optimize_fun(n_layers, loss_fun)
            print("#### LAYERS: {}, LOSS: {}: MODEL: {}, EVAL: {} ####\n".format(n_layers, loss_fun, model, eval))
            best_results[n_layers, loss_fun] = {
                "model": model,
                "eval": eval,
            }


In [ ]:
# RUN
optimize_predictor(_predictor_sa_optimization)

Epoch 1/10
18/18 [==============================] - 1s 17ms/step - loss: 0.4058 - accuracy: 0.9023 - val_loss: 0.1723 - val_accuracy: 1.0000
Epoch 2/10
18/18 [==============================] - 0s 5ms/step - loss: 0.1308 - accuracy: 1.0000 - val_loss: 0.0916 - val_accuracy: 1.0000
Epoch 3/10
18/18 [==============================] - 0s 6ms/step - loss: 0.1032 - accuracy: 1.0000 - val_loss: 0.0829 - val_accuracy: 1.0000
Epoch 4/10
18/18 [==============================] - 0s 6ms/step - loss: 0.0951 - accuracy: 1.0000 - val_loss: 0.0762 - val_accuracy: 1.0000
Epoch 5/10
18/18 [==============================] - 0s 6ms/step - loss: 0.0897 - accuracy: 1.0000 - val_loss: 0.0762 - val_accuracy: 1.0000
Epoch 6/10
18/18 [==============================] - 0s 6ms/step - loss: 0.0876 - accuracy: 1.0000 - val_loss: 0.0778 - val_accuracy: 1.0000
Epoch 7/10
18/18 [==============================] - 0s 5ms/step - loss: 0.0890 - accuracy: 1.0000 - val_loss: 0.0814 - val_accuracy: 1.0000
Epoch 8/10
18/18 [=